In [1]:
!pip install -U pathway pandas numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 11.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 122.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 82

In [1]:
import pathway as pw
import pandas as pd
import numpy as np
from datetime import timedelta

In [10]:
BASE_PRICE = 10.0
ALPHA = 2.0
PRICE_MIN = 5.0
PRICE_MAX = 20.0

def linear_update(prev_price, occupancy, capacity):
    util = occupancy / capacity
    return float(np.clip(prev_price + ALPHA * util, MIN_P, MAX_P))


In [11]:
df = pd.read_csv('dataset.csv')
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')
df = df.sort_values(by=['SystemCodeNumber','Timestamp']).reset_index(drop=True)

In [13]:
df['Price'] = np.nan

# Step 4: Apply Model 1 pricing logic per lot
# Group by Lot ID
for lot_id, group in df.groupby('SystemCodeNumber'):
    lot_df = group.copy()
    prices = [BASE_PRICE]  # Start with base price

    # Loop over time records starting from second row
    for i in range(1, len(lot_df)):
        occupancy = lot_df.iloc[i-1]['Occupancy']
        capacity = lot_df.iloc[i-1]['Capacity']
        prev_price = prices[-1]

        # Avoid divide-by-zero
        occ_ratio = occupancy / capacity if capacity > 0 else 0
        new_price = prev_price + ALPHA * occ_ratio

        # Clip to bounds
        new_price = np.clip(new_price, PRICE_MIN, PRICE_MAX)
        prices.append(new_price)

    # Set prices back into dataframe
    df.loc[lot_df.index, 'Price'] = prices

# Optional: format price to 2 decimals
df['Price'] = df['Price'].round(2)
sample_lot = df[df['SystemCodeNumber'] == 'BHMBCCMKT01'][['SystemCodeNumber', 'Timestamp', 'Occupancy', 'Capacity', 'Price']]
print(sample_lot.head(10))

  SystemCodeNumber           Timestamp  Occupancy  Capacity  Price
0      BHMBCCMKT01 2016-10-04 07:59:00         61       577  10.00
1      BHMBCCMKT01 2016-10-04 08:25:00         64       577  10.21
2      BHMBCCMKT01 2016-10-04 08:59:00         80       577  10.43
3      BHMBCCMKT01 2016-10-04 09:32:00        107       577  10.71
4      BHMBCCMKT01 2016-10-04 09:59:00        150       577  11.08
5      BHMBCCMKT01 2016-10-04 10:26:00        177       577  11.60
6      BHMBCCMKT01 2016-10-04 10:59:00        219       577  12.21
7      BHMBCCMKT01 2016-10-04 11:25:00        247       577  12.97
8      BHMBCCMKT01 2016-10-04 11:59:00        259       577  13.83
9      BHMBCCMKT01 2016-10-04 12:29:00        266       577  14.73


In [19]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.layouts import column
from bokeh.palettes import Category10
import pandas as pd

output_notebook()

# Assume df is your DataFrame with these columns:
# 'SystemCodeNumber', 'Timestamp', 'Price'

# Choose top 3 most active lots
top_lots = df['SystemCodeNumber'].value_counts().nlargest(3).index.tolist()

plots = []
colors = Category10[3]

for i, lot_id in enumerate(top_lots):
    lot_df = df[df['SystemCodeNumber'] == lot_id]
    source = ColumnDataSource(data={
        'x': lot_df['Timestamp'],
        'y': lot_df['Price']
    })

    p = figure(title=f"Price Evolution - {lot_id}", x_axis_type='datetime', width=700, height=300)
    p.line('x', 'y', source=source, line_width=2, color=colors[i], legend_label=lot_id)
    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Price "
    p.legend.location = "top_right"
    plots.append(p)

show(column(*plots))